# 1_ Import dependencies

In [1]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

In [2]:
#! pip install stable-baselines3[extra]

In [3]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

C:\path\to\anaconda3\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


# 2_ Load environment

In [4]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:15.0
Episode:2 Score:13.0
Episode:3 Score:16.0
Episode:4 Score:27.0
Episode:5 Score:23.0


In [6]:
env.render()

True

In [7]:
#Différentes actions possibles dans l'environnement
print(env.action_space)
print(env.action_space.sample())

Discrete(2)
1



## Understanding The Environment

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py


In [8]:
env.observation_space.sample()

array([ 2.7914999e+00, -3.7556725e+37,  1.6609731e-01,  2.2316094e+38],
      dtype=float32)

# 3_ Train an RL model

In [9]:
import os
log_path = os.path.join('./training', 'logs')

env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cpu device


In [10]:
#PPO??

In [11]:
model.learn(total_timesteps=20000)

Exception ignored in: <function Viewer.__del__ at 0x0000003C76DDF3A0>
Traceback (most recent call last):
  File "C:\path\to\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "C:\path\to\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "C:\path\to\anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 328, in close
    super(Win32Window, self).close()
  File "C:\path\to\anaconda3\lib\site-packages\pyglet\window\__init__.py", line 857, in close
    app.windows.remove(self)
  File "C:\path\to\anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000003C6D476180; to 'Win32Window' at 0x0000003C6D46BD00>


Logging to ./training\logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 311  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 419         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009145913 |
|    clip_fraction        | 0.105       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00212     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.19        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0166     |
|    value_loss           | 59.8        |
-----------------------------------------
-

# 4_ Save and reload model 

In [12]:
PPO_path = os.path.join('./training', 'saved_models', 'PPO_model_CartPole')

In [13]:
model.save(PPO_path)

In [14]:
del model

In [15]:
model = PPO.load(PPO_path, env=env)

# 5_ Evaluation

In [16]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\path\to\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [17]:
env.close()

# 6_ Testing

In [19]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _state = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[148.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


# 7_ Viewing logs in tensorboard

In [20]:
training_log_path = os.path.join(log_path, 'PPO_4')

In [21]:
!tensorboard --logdir={training_log_path}

^C


# 8_ Adding a callback to the training stage

In [22]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [23]:
save_path = os.path.join('training', 'saved_models')
log_path = os.path.join('training', 'logs')

In [24]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [25]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [26]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [27]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training\logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 1327 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 955         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008753135 |
|    clip_fraction        | 0.0948      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000804    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.47        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.014      |
|    value_loss           | 51.1        |
-----------------------------------------
---

In [28]:
model_path = os.path.join('training', 'saved_models', 'best_model')
model = PPO.load(model_path, env=env)


In [29]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [30]:
env.close()

# 9_ Changing policies

In [31]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [32]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [33]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 1144 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 704         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015209224 |
|    clip_fraction        | 0.269       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.68       |
|    explained_variance   | -0.00992    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.08        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0293     |
|    value_loss           | 21.5        |
-----------------------------------------
----------------------------------

# 10_ Changing Algo

In [34]:
from stable_baselines3 import DQN

In [35]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [36]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training\logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration rate | 0.954    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 601      |
|    time_elapsed     | 0        |
|    total timesteps  | 96       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.915    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 789      |
|    time_elapsed     | 0        |
|    total timesteps  | 178      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.865    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 1010     |
|    time_elapsed     | 0        |
|    total timesteps  | 284      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 1665     |
|    time_elapsed     | 1        |
|    total timesteps  | 2497     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 1671     |
|    time_elapsed     | 1        |
|    total timesteps  | 2559     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 1691     |
|    time_elapsed     | 1        |
|    total timesteps  | 2647     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 1851     |
|    time_elapsed     | 2        |
|    total timesteps  | 4832     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 1847     |
|    time_elapsed     | 2        |
|    total timesteps  | 4908     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 1870     |
|    time_elapsed     | 2        |
|    total timesteps  | 5051     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 1952     |
|    time_elapsed     | 3        |
|    total timesteps  | 7094     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 1952     |
|    time_elapsed     | 3        |
|    total timesteps  | 7157     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 1959     |
|    time_elapsed     | 3        |
|    total timesteps  | 7249     |
----------------------------------
----------------------------------
| rollout/          

Eval num_timesteps=9520, episode_reward=9.60 +/- 0.49
Episode length: 9.60 +/- 0.49
----------------------------------
| eval/               |          |
|    mean_ep_length   | 9.6      |
|    mean_reward      | 9.6      |
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    total timesteps  | 9520     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 1945     |
|    time_elapsed     | 4        |
|    total timesteps  | 9604     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 1953     |
|    time_elapsed     | 5        |
|    total timesteps  | 9773     |
------

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 1972     |
|    time_elapsed     | 6        |
|    total timesteps  | 11935    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 1973     |
|    time_elapsed     | 6        |
|    total timesteps  | 12011    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 1979     |
|    time_elapsed     | 6        |
|    total timesteps  | 12118    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 2028     |
|    time_elapsed     | 7        |
|    total timesteps  | 14303    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 2028     |
|    time_elapsed     | 7        |
|    total timesteps  | 14402    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 2026     |
|    time_elapsed     | 7        |
|    total timesteps  | 14483    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 2037     |
|    time_elapsed     | 8        |
|    total timesteps  | 16635    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 2040     |
|    time_elapsed     | 8        |
|    total timesteps  | 16728    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 2042     |
|    time_elapsed     | 8        |
|    total timesteps  | 16805    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 2074     |
|    time_elapsed     | 9        |
|    total timesteps  | 19163    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 2073     |
|    time_elapsed     | 9        |
|    total timesteps  | 19224    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 2074     |
|    time_elapsed     | 9        |
|    total timesteps  | 19313    |
----------------------------------
----------------------------------
| rollout/          

In [37]:
dqn_path = os.path.join('training', 'saved_models', 'DQN_model')

In [38]:
model.save(dqn_path)

In [39]:
model = DQN.load(dqn_path, env=env)

In [40]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.3, 0.45825756949558405)

In [41]:
env.close()